In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle
import shap
import lime.lime_tabular
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os


# Load your data (assuming output.csv is your dataset)
data = pd.read_csv("output.csv")
data = data.drop_duplicates()

# Prepare your features and target (adjust columns as needed)
X = data.drop(columns=['gravity', 'AccID', 'vehicleID', 'num_veh'], errors='ignore')
y = data['gravity']

# Split data (this is optional if you only need the final model)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train your model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save the trained model
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

# Optionally, save your test data if you want to use it in Streamlit later
X_test.to_csv("X_test.csv", index=False)


In [4]:
# Ensure test data is loaded
X_test = pd.read_csv("X_test.csv")  # Load the test dataset if not already loaded

# Create a smaller sample for SHAP (to reduce computation time)
X_sample = X_test.sample(n=50, random_state=42)  # Define X_sample here

# Create SHAP Explainer
explainer = shap.TreeExplainer(model)

# Compute SHAP values
shap_values = explainer.shap_values(X_sample)  # Now X_sample is correctly defined

# Determine the correct class index for SHAP dependence plot
class_index = 0  # Change this index to visualize other classes (0, 1, 2, 3)

# Print SHAP value shapes to check correctness
for i, sv in enumerate(shap_values):
    print(f"Class {i}: {sv.shape}")

# SHAP Summary Plot
# Ensure shap_values matches the feature shape of X_sample
shap_values_fixed = shap_values[class_index]  

# Handle potential extra columns in shap_values
if shap_values_fixed.shape[1] != X_sample.shape[1]:
    shap_values_fixed = shap_values_fixed[:, :X_sample.shape[1]]

shap.summary_plot(shap_values_fixed, X_sample, show=False)
fig_summary = plt.gcf()

# SHAP Dependence Plot with correct class selection
feature_name = X_sample.columns[0]
shap.dependence_plot(feature_name, shap_values[class_index], X_sample, show=False)
fig_dependence = plt.gcf()

# LIME Explanation for a single prediction
explainer_lime = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train.values,
    feature_names=X_train.columns,
    class_names=['Class 0', 'Class 1', 'Class 2', 'Class 3'],  # Adjust class names as needed
    mode='classification'
)

exp = explainer_lime.explain_instance(X_test.iloc[0], model.predict_proba)
fig_lime = exp.as_pyplot_figure()

# Display results
plt.show(fig_summary)
plt.show(fig_dependence)
plt.show(fig_lime)


Class 0: (37, 4)
Class 1: (37, 4)
Class 2: (37, 4)
Class 3: (37, 4)
Class 4: (37, 4)
Class 5: (37, 4)
Class 6: (37, 4)
Class 7: (37, 4)
Class 8: (37, 4)
Class 9: (37, 4)
Class 10: (37, 4)
Class 11: (37, 4)
Class 12: (37, 4)
Class 13: (37, 4)
Class 14: (37, 4)
Class 15: (37, 4)
Class 16: (37, 4)
Class 17: (37, 4)
Class 18: (37, 4)
Class 19: (37, 4)
Class 20: (37, 4)
Class 21: (37, 4)
Class 22: (37, 4)
Class 23: (37, 4)
Class 24: (37, 4)
Class 25: (37, 4)
Class 26: (37, 4)
Class 27: (37, 4)
Class 28: (37, 4)
Class 29: (37, 4)
Class 30: (37, 4)
Class 31: (37, 4)
Class 32: (37, 4)
Class 33: (37, 4)
Class 34: (37, 4)
Class 35: (37, 4)
Class 36: (37, 4)
Class 37: (37, 4)
Class 38: (37, 4)
Class 39: (37, 4)
Class 40: (37, 4)
Class 41: (37, 4)
Class 42: (37, 4)
Class 43: (37, 4)
Class 44: (37, 4)
Class 45: (37, 4)
Class 46: (37, 4)
Class 47: (37, 4)
Class 48: (37, 4)
Class 49: (37, 4)


AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.